In [8]:
from langchain_core.messages import AIMessage, HumanMessage


chats = {
    1: [
        HumanMessage(content="Translate from English to French: I love programming."),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ],
    2: [
        HumanMessage(content="Tell me about the Major Cities"),
        AIMessage(content="It's NewYork, Paris, Bejing, Hongkong"),
        HumanMessage(content="What did you just say?"),
    ],
}

print(chats)


{1: [HumanMessage(content='Translate from English to French: I love programming.', additional_kwargs={}, response_metadata={}), AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={}), HumanMessage(content='What did you just say?', additional_kwargs={}, response_metadata={})], 2: [HumanMessage(content='Tell me about the Major Cities', additional_kwargs={}, response_metadata={}), AIMessage(content="It's NewYork, Paris, Bejing, Hongkong", additional_kwargs={}, response_metadata={}), HumanMessage(content='What did you just say?', additional_kwargs={}, response_metadata={})]}


In [3]:
# multi_agent/QnA_Agent/agent.py
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_react_agent, AgentExecutor, Tool
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain import hub

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


def qna_agent_response(user_input: str) -> str:
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

    search_tool = DuckDuckGoSearchRun()
    tool = Tool(
        name="Search",
        func=search_tool.run,
        description="Use this tool to look up information on the web ONLY when needed.",
    )

    prompt = hub.pull("hwchase17/react-chat")

    agent = create_react_agent(llm=llm, tools=[tool], prompt=prompt)

    agent_executor = AgentExecutor(
        agent=agent,
        tools=[tool],
        memory=memory,
        verbose=True,
        handle_parsing_errors=True,
    )

    result = agent_executor.invoke({"input": user_input})
    return result["output"]


query = input("Enter query : ")

result = qna_agent_response(query)

print(result)

print(memory)


c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\AppData\Local\Temp\ipykernel_14048\2966365582.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hi there! How can I help you today?

> Finished chain.
Hi there! How can I help you today?
chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]) return_messages=True memory_key='chat_history'


In [6]:
memory.chat_memory.messages

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]

In [21]:
chats[list(chats.keys())[-1] + 1] = memory.chat_memory.messages

chats

{1: [HumanMessage(content='Translate from English to French: I love programming.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What did you just say?', additional_kwargs={}, response_metadata={})],
 2: [HumanMessage(content='Tell me about the Major Cities', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It's NewYork, Paris, Bejing, Hongkong", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What did you just say?', additional_kwargs={}, response_metadata={})],
 3: [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={})]}